In [2]:
import cv2
from ultralytics import YOLO
import os
import json
from glob import glob
from matplotlib import pyplot as plt

In [2]:
def get_item_from_img(img_path, model):
    img_cv2 = cv2.imread(img_path)
    img_id = os.path.splitext(os.path.basename(img_path))[0]

    results = model.predict(source=img_cv2, conf=0.5, half=True)

    hist = f"C:/Users/Admin/OneDrive - Hochiminh City University of Education/Documents/Y3_TXA_NDTN/YOLO/bovw/hist/{img_id}_hist.txt"

    objs = []

    if results[0].boxes is None or len(results[0].boxes) == 0:
        return None

    print (len(results[0].boxes))
    for track_id, box in enumerate(results[0].boxes):
        print(f"Processing {img_id} - {track_id + 1}...")
        class_label = results[0].names[int(box.cls)]

        obj_id = f"{img_id}_{track_id + 1}"

        obj_info = {
            "ObjID": obj_id,
            "ImgID": img_id,
            "Class": class_label,
            "Feature": hist
        }
        objs.append(obj_info)

    return objs

In [3]:
import time
import gc

In [6]:
def save_obj_img_to_bovw(img_files, model):
    batch_size = 1000
    for start_index in range(0, len(img_files), batch_size):
        end_index = min(start_index + batch_size, len(img_files))
        batch_files = img_files[start_index:end_index]
        results = []
        for img_path in batch_files:
            print(f"Processing {img_path}...")
            objs = get_item_from_img(img_path, model)
            if objs:
                results.extend(objs)

        time.sleep(0.1)

        # Process results or save to file
        print(f"Processed batch {start_index // batch_size + 1}")
        
        json_file_path = "./bovw/new_json/list_bovw_newsmol.json"
        if os.path.exists(json_file_path):
            with open(json_file_path, "r") as f:
                try:
                    existing_data = json.load(f)
                except json.JSONDecodeError:
                    existing_data = []
        else:
            existing_data = []

        # Append new results to existing data
        existing_data.extend(results)

        with open(json_file_path, "w") as f:
            json.dump(existing_data, f, indent=4)

        # Clear memory
        del results
        gc.collect()

In [5]:
model = YOLO("yolov8x-oiv7")


video_path = "./video/Close to gray.mp4"
cv2_video = cv2.VideoCapture(video_path)

ret = True

while ret:
    ret, frame = cv2_video.read()
    if ret:
        results = model.track(source=frame, conf=0.5, persist=True)

        fname_ = results[0].plot()

        cv2.imshow("Frame", fname_)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cv2_video.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 63.7ms
Speed: 8.9ms preprocess, 63.7ms inference, 37.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 49.2ms
Speed: 3.5ms preprocess, 49.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 50.2ms
Speed: 3.0ms preprocess, 50.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 49.3ms
Speed: 2.0ms preprocess, 49.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 49.6ms
Speed: 2.0ms preprocess, 49.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 45.9ms
Speed: 2.0ms preprocess, 45.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 47.1ms
Speed: 2.4ms preprocess, 47.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 46.2ms
Speed: 3.0ms preprocess, 46.2ms 

RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


: 

In [8]:
save_obj_img_to_bovw(sub_img_files, model)

Processing ./img/resize\0000253ea4ecbf19.jpg...

0: 480x640 (no detections), 148.0ms
Speed: 6.0ms preprocess, 148.0ms inference, 55.0ms postprocess per image at shape (1, 3, 480, 640)
Processing ./img/resize\0000286a5c6a3eb5.jpg...

0: 480x640 1 Footwear, 1 Man, 36.0ms
Speed: 1.0ms preprocess, 36.0ms inference, 129.0ms postprocess per image at shape (1, 3, 480, 640)
2
Processing 0000286a5c6a3eb5 - 1...
Processing 0000286a5c6a3eb5 - 2...
Processing ./img/resize\00003bfccf5f36c2.jpg...

0: 480x640 (no detections), 36.0ms
Speed: 1.0ms preprocess, 36.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Processing ./img/resize\000045257f66b9e2.jpg...

0: 480x640 5 Mans, 7 Womans, 36.0ms
Speed: 1.0ms preprocess, 36.0ms inference, 22.0ms postprocess per image at shape (1, 3, 480, 640)
12
Processing 000045257f66b9e2 - 1...
Processing 000045257f66b9e2 - 2...
Processing 000045257f66b9e2 - 3...
Processing 000045257f66b9e2 - 4...
Processing 000045257f66b9e2 - 5...
Processing 000045

RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [49]:
with open("./bovw/json/bovw_class11.json", "w") as f:
    json.dump(list_bovw, f, indent=4)

In [27]:
img_path = "C:/Users/Admin/fiftyone/open-images-v7/train/data/0207ea1877c62bf1.jpg"

img_cv2 = cv2.imread(img_path)

results = model(source=img_cv2, conf=0.5)
print(results)


0: 256x640 (no detections), 349.9ms
Speed: 4.1ms preprocess, 349.9ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 640)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'Accordion', 1: 'Adhesive tape', 2: 'Aircraft', 3: 'Airplane', 4: 'Alarm clock', 5: 'Alpaca', 6: 'Ambulance', 7: 'Animal', 8: 'Ant', 9: 'Antelope', 10: 'Apple', 11: 'Armadillo', 12: 'Artichoke', 13: 'Auto part', 14: 'Axe', 15: 'Backpack', 16: 'Bagel', 17: 'Baked goods', 18: 'Balance beam', 19: 'Ball', 20: 'Balloon', 21: 'Banana', 22: 'Band-aid', 23: 'Banjo', 24: 'Barge', 25: 'Barrel', 26: 'Baseball bat', 27: 'Baseball glove', 28: 'Bat (Animal)', 29: 'Bathroom accessory', 30: 'Bathroom cabinet', 31: 'Bathtub', 32: 'Beaker', 33: 'Bear', 34: 'Bed', 35: 'Bee', 36: 'Beehive', 37: 'Beer', 38: 'Beetle', 39: 'Bell pepper', 40: 'Belt', 41: 'Bench', 42: 'Bicycle', 43: 'Bicycle helmet', 44: 'Bicycle wheel', 45: 'Bidet',

In [14]:
for bovw in list_bovw:
    if bovw["Class"] == "Accordion":
        print(bovw)

{'ObjID': '04dd8510015ed600_1', 'ImgID': '04dd8510015ed600', 'Class': 'Accordion', 'Feature': 'C:/Users/Admin/OneDrive - Hochiminh City University of Education/Documents/Y3_TXA_NDTN/YOLO/bovw/hist/04dd8510015ed600_hist.txt'}
{'ObjID': '0c37b2fb66423018_122', 'ImgID': '0c37b2fb66423018', 'Class': 'Accordion', 'Feature': 'C:/Users/Admin/OneDrive - Hochiminh City University of Education/Documents/Y3_TXA_NDTN/YOLO/bovw/hist/0c37b2fb66423018_hist.txt'}
{'ObjID': '0c37b2fb66423018_147', 'ImgID': '0c37b2fb66423018', 'Class': 'Accordion', 'Feature': 'C:/Users/Admin/OneDrive - Hochiminh City University of Education/Documents/Y3_TXA_NDTN/YOLO/bovw/hist/0c37b2fb66423018_hist.txt'}
{'ObjID': '0c37b2fb66423018_172', 'ImgID': '0c37b2fb66423018', 'Class': 'Accordion', 'Feature': 'C:/Users/Admin/OneDrive - Hochiminh City University of Education/Documents/Y3_TXA_NDTN/YOLO/bovw/hist/0c37b2fb66423018_hist.txt'}
{'ObjID': '0c37b2fb66423018_199', 'ImgID': '0c37b2fb66423018', 'Class': 'Accordion', 'Feature'

In [21]:
img_paths = glob("C:/Users/Admin/fiftyone/open-images-v7/train/data/*.jpg")
list_objs = []

for img_path in img_paths[0:10]:
    objs = get_item_from_img(img_path, model)
    if objs is not None:
        list_objs.extend(objs)

print(list_objs)


0: 480x640 (no detections), 386.4ms
Speed: 718.9ms preprocess, 386.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 448x640 1 Footwear, 1 Man, 72.2ms
Speed: 7.0ms preprocess, 72.2ms inference, 3.6ms postprocess per image at shape (1, 3, 448, 640)

0: 640x480 (no detections), 86.5ms
Speed: 0.0ms preprocess, 86.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)

0: 480x640 7 Mans, 9 Womans, 80.7ms
Speed: 3.5ms preprocess, 80.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 640x384 1 Toy, 64.1ms
Speed: 0.0ms preprocess, 64.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 448x640 1 Car, 74.4ms
Speed: 6.0ms preprocess, 74.4ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)

0: 480x640 4 Persons, 83.2ms
Speed: 2.4ms preprocess, 83.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 448x640 1 Window, 71.8ms
Speed: 0.0ms preprocess, 71.8ms inference, 8.0ms postproces

In [1]:
import torch

In [2]:
torch.version.cuda

'11.8'

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.current_device()

0

In [7]:
torch.cuda.device(0)
torch.cuda.device_count()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3050 Laptop GPU'